In [1]:
import sys
import os
os.chdir("../")

In [2]:
import numpy as np
import pandas as pd
import data
import preprocess_utils.session2vec as sess2vec
from preprocess_utils.last_clickout_indices import find as find_last_clickout_indices
from tqdm.auto import tqdm
tqdm.pandas()
from IPython.display import display
pd.options.display.max_columns = None

Using TensorFlow backend.


In [3]:
train_df = data.train_df('small', 'cluster_recurrent')

In [4]:
train_df.head(30)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,3.0
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
5,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
6,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
7,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
8,00RL8Z82B2Z1,aff3928535f48,1541037542,9,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,5.0
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,1.0
14,00RL8Z82B2Z1,aff3928535f48,1541038469,15,search for poi,Surry Hills,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
15,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031...,162|25|150|143|101|49|118|131|18|100|101|143|5...,1.0


In [5]:
indices = find_last_clickout_indices(train_df)

In [89]:
res_df = train_df[['user_id','session_id','reference','impressions']].loc[indices]

In [90]:
res_df.head()

,user_id,session_id,reference,impressions
27778,0004IOZI7CKF,0146f7cb014ba,110985,2632452|5747484|110985|84220|3752262|85103|852...
464545,0008B0X0HC39,1b703eda68d85,2762974,2762974
697007,000ZN9219G41,1086064207e4f,149015,149015|1077638|34819|4384770|41629|5525034|348...
884528,001OCXQ9PXET,362dc2be48010,2450306,2450306|1473305|6398796|1161413|13435|231736|1...
725484,001TEVEVUEBE,394f5ad9aa596,8415608,3366614|2670592|6625742|2881004|5708270|841560...


In [97]:
def add_label(row):
    impress = list(map(int, row['impressions'].split('|')))
    ref = row.reference
    
    if pd.isnull(ref):
        return np.NaN
    
    if ref in impress:
        return impress.index(ref)
    else:
        return 0

In [98]:
res_df['label'] = res_df.astype({'reference':'float'}).progress_apply(add_label, axis=1)

In [99]:
res_df.dropna()

,user_id,session_id,reference,impressions,label
27778,0004IOZI7CKF,0146f7cb014ba,110985,2632452|5747484|110985|84220|3752262|85103|852...,2
464545,0008B0X0HC39,1b703eda68d85,2762974,2762974,0
697007,000ZN9219G41,1086064207e4f,149015,149015|1077638|34819|4384770|41629|5525034|348...,0
884528,001OCXQ9PXET,362dc2be48010,2450306,2450306|1473305|6398796|1161413|13435|231736|1...,0
725484,001TEVEVUEBE,394f5ad9aa596,8415608,3366614|2670592|6625742|2881004|5708270|841560...,5
450143,002IXCPKK20W,dcdc75a82f142,9772462,2730178|2829436|2551186|9772462|5092274|969192...,3
368440,002O8H8GYKSQ,ab252cd2bafd3,3392402,3469158|3392402|1394202|107597|4085894|106755|...,1
647663,002Z7VZFWM79,d4cc248f60752,56542,56542|79374|3132582|72594|68177|72865|63704|62...,0
450147,003728BB2ALE,eaf4b65f13e8e,10875780,10875780|8784374|3758182|3766642|5058396|99548...,0
203387,003CQCD8POV7,4c2ae42150f97,3139558,3139558|2499058|7252818|8117350|3994174|997034...,0


In [8]:
from extract_features.rnn.session_label import SessionLabel
f = SessionLabel('small').read_feature()

session_label feature read


In [9]:
f

,user_id,session_id,label
27778,0004IOZI7CKF,0146f7cb014ba,2
464545,0008B0X0HC39,1b703eda68d85,0
697007,000ZN9219G41,1086064207e4f,0
884528,001OCXQ9PXET,362dc2be48010,0
725484,001TEVEVUEBE,394f5ad9aa596,5
450143,002IXCPKK20W,dcdc75a82f142,3
368440,002O8H8GYKSQ,ab252cd2bafd3,1
647663,002Z7VZFWM79,d4cc248f60752,0
450147,003728BB2ALE,eaf4b65f13e8e,0
203387,003CQCD8POV7,4c2ae42150f97,0


In [11]:
(f.label == 0) * 1

27778     0
464545    1
697007    1
884528    1
725484    0
450143    0
368440    0
647663    1
450147    1
203387    1
684003    0
580734    0
738136    1
412180    1
580767    0
464550    0
277978    0
109933    1
659828    0
368442    0
134330    0
569455    1
505945    0
697008    0
773447    1
670539    0
53927     0
438449    1
344217    1
190142    1
         ..
438447    1
897134    0
425556    0
670504    0
397802    1
289145    0
869776    1
438448    0
618711    1
696987    0
738135    1
121881    0
897138    1
841942    1
696996    0
855482    0
530281    1
356699    0
555572    0
478929    0
289146    1
761012    1
683999    0
478931    1
13027     0
397804    0
580728    0
397819    0
263155    1
761013    0
Name: label, Length: 37664, dtype: int64

## Add reference label to df

In [11]:
t = train_df.sort_values(['user_id','session_id','timestamp','step'])
df = t.loc[indices].merge(f.drop(['user_id','session_id'],axis=1), how='left', left_index=True, right_index=True)

In [12]:
df

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence,label
27778,0004IOZI7CKF,0146f7cb014ba,1541266868,5,clickout item,110985,DE,"Valencia, Spain",tablet,NaN,2632452|5747484|110985|84220|3752262|85103|852...,75|62|113|91|106|115|75|68|81|86|36|66|86|159|...,1.0,2
464545,0008B0X0HC39,1b703eda68d85,1541445203,1,clickout item,2762974,US,"Coron, Philippines",desktop,NaN,2762974,494,1.0,0
697007,000ZN9219G41,1086064207e4f,1541378829,11,clickout item,149015,CA,"Lisbon, Portugal",mobile,Air Conditioning|Car Park|Sort by Price|Hotel|...,149015|1077638|34819|4384770|41629|5525034|348...,52|60|64|66|70|70|71|72|75|76|77|80|80|84|84|8...,1.0,0
884528,001OCXQ9PXET,362dc2be48010,1541102228,4,clickout item,2450306,NL,"Marne-la-Vallée, France",mobile,Very Good Rating|Focus on Distance,2450306|1473305|6398796|1161413|13435|231736|1...,67|84|65|79|103|74|76|98|73|98|62|55|121|74|92...,1.0,0
725484,001TEVEVUEBE,394f5ad9aa596,1541049172,1,clickout item,8415608,MX,"Atlixco, Mexico",desktop,NaN,3366614|2670592|6625742|2881004|5708270|841560...,61|80|102|87|126|85|26|81|50,1.0,5
450143,002IXCPKK20W,dcdc75a82f142,1541337061,2,clickout item,9772462,BR,"Bertioga, Brazil",mobile,NaN,2730178|2829436|2551186|9772462|5092274|969192...,84|71|79|30|34|45|68|273|62|35,1.0,3
368440,002O8H8GYKSQ,ab252cd2bafd3,1541328734,2,clickout item,3392402,NL,"Veenendaal, Netherlands",desktop,NaN,3469158|3392402|1394202|107597|4085894|106755|...,136|92|92|74|90|100|73|93|145|68|114|123|148|1...,1.0,1
647663,002Z7VZFWM79,d4cc248f60752,1541039434,1,clickout item,56542,US,"Galveston, USA",mobile,NaN,56542|79374|3132582|72594|68177|72865|63704|62...,140|114|52|86|140|67|141|53|87|60|38|49|61|43|...,1.0,0
450147,003728BB2ALE,eaf4b65f13e8e,1541338113,4,clickout item,10875780,IN,"Allahabad, India",mobile,Sort By Rating,10875780|8784374|3758182|3766642|5058396|99548...,17|31|43|107|41|22|30|36|25|24|36|36|28|27|31|...,1.0,0
203387,003CQCD8POV7,4c2ae42150f97,1541366623,1,clickout item,3139558,BR,"Tatuí, Brazil",desktop,NaN,3139558|2499058|7252818|8117350|3994174|997034...,56|120|102|108|157|50|117,1.0,0


In [13]:
from sklearn.preprocessing import OneHotEncoder

In [14]:
enc = OneHotEncoder(n_values=25, sparse=False)

In [15]:
enc.fit_transform(df['label'].values.reshape(-1, 1))

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:331: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)]'.
  warnings.warn(msg, DeprecationWarning)


array([[0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [109]:
df

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence,label
27774,0004IOZI7CKF,0146f7cb014ba,1541266717,1,search for destination,"Valencia, Spain",DE,"Valencia, Spain",tablet,NaN,NaN,NaN,1.0,NaN
27775,0004IOZI7CKF,0146f7cb014ba,1541266769,2,interaction item info,3381482,DE,"Valencia, Spain",tablet,NaN,NaN,NaN,1.0,NaN
27776,0004IOZI7CKF,0146f7cb014ba,1541266796,3,clickout item,2627602,DE,"Valencia, Spain",tablet,NaN,110976|88759|150904|3549934|4084196|2094980|11...,70|83|150|143|86|74|78|96|55|72|117|84|92|56|1...,1.0,NaN
27777,0004IOZI7CKF,0146f7cb014ba,1541266841,4,clickout item,7822344,DE,"Valencia, Spain",tablet,NaN,110976|88759|150904|3549934|4084196|2094980|11...,70|83|150|143|86|74|78|96|55|72|117|84|92|56|1...,1.0,NaN
27778,0004IOZI7CKF,0146f7cb014ba,1541266868,5,clickout item,110985,DE,"Valencia, Spain",tablet,NaN,2632452|5747484|110985|84220|3752262|85103|852...,75|62|113|91|106|115|75|68|81|86|36|66|86|159|...,1.0,2.0
464545,0008B0X0HC39,1b703eda68d85,1541445203,1,clickout item,2762974,US,"Coron, Philippines",desktop,NaN,2762974,494,1.0,0.0
696997,000ZN9219G41,1086064207e4f,1541378764,1,filter selection,Air Conditioning,CA,"Lisbon, Portugal",mobile,Air Conditioning,NaN,NaN,1.0,NaN
696998,000ZN9219G41,1086064207e4f,1541378766,2,filter selection,Car Park,CA,"Lisbon, Portugal",mobile,Air Conditioning|Car Park,NaN,NaN,1.0,NaN
696999,000ZN9219G41,1086064207e4f,1541378776,3,change of sort order,interaction sort button,CA,"Lisbon, Portugal",mobile,NaN,NaN,NaN,1.0,NaN
697000,000ZN9219G41,1086064207e4f,1541378786,4,change of sort order,price only,CA,"Lisbon, Portugal",mobile,NaN,NaN,NaN,1.0,NaN


In [6]:
sess2vec.add_reference_labels(train_df.loc[indices], mode='small')

session_label feature read


/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:331: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)]'.
  warnings.warn(msg, DeprecationWarning)


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence,label,ref_0,ref_1,ref_2,ref_3,ref_4,ref_5,ref_6,ref_7,ref_8,ref_9,ref_10,ref_11,ref_12,ref_13,ref_14,ref_15,ref_16,ref_17,ref_18,ref_19,ref_20,ref_21,ref_22,ref_23,ref_24
27778,0004IOZI7CKF,0146f7cb014ba,1541266868,5,clickout item,110985,DE,"Valencia, Spain",tablet,NaN,2632452|5747484|110985|84220|3752262|85103|852...,75|62|113|91|106|115|75|68|81|86|36|66|86|159|...,1.0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
464545,0008B0X0HC39,1b703eda68d85,1541445203,1,clickout item,2762974,US,"Coron, Philippines",desktop,NaN,2762974,494,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
697007,000ZN9219G41,1086064207e4f,1541378829,11,clickout item,149015,CA,"Lisbon, Portugal",mobile,Air Conditioning|Car Park|Sort by Price|Hotel|...,149015|1077638|34819|4384770|41629|5525034|348...,52|60|64|66|70|70|71|72|75|76|77|80|80|84|84|8...,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
884528,001OCXQ9PXET,362dc2be48010,1541102228,4,clickout item,2450306,NL,"Marne-la-Vallée, France",mobile,Very Good Rating|Focus on Distance,2450306|1473305|6398796|1161413|13435|231736|1...,67|84|65|79|103|74|76|98|73|98|62|55|121|74|92...,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
725484,001TEVEVUEBE,394f5ad9aa596,1541049172,1,clickout item,8415608,MX,"Atlixco, Mexico",desktop,NaN,3366614|2670592|6625742|2881004|5708270|841560...,61|80|102|87|126|85|26|81|50,1.0,5,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
450143,002IXCPKK20W,dcdc75a82f142,1541337061,2,clickout item,9772462,BR,"Bertioga, Brazil",mobile,NaN,2730178|2829436|2551186|9772462|5092274|969192...,84|71|79|30|34|45|68|273|62|35,1.0,3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
368440,002O8H8GYKSQ,ab252cd2bafd3,1541328734,2,clickout item,3392402,NL,"Veenendaal, Netherlands",desktop,NaN,3469158|3392402|1394202|107597|4085894|106755|...,136|92|92|74|90|100|73|93|145|68|114|123|148|1...,1.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
647663,002Z7VZFWM79,d4cc248f60752,1541039434,1,clickout item,56542,US,"Galveston, USA",mobile,NaN,56542|79374|3132582|72594|68177|72865|63704|62...,140|114|52|86|140|67|141|53|87|60|38|49|61|43|...,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
450147,003728BB2ALE,eaf4b65f13e8e,1541338113,4,clickout item,10875780,IN,"Allahabad, India",mobile,Sort By Rating,10875780|8784374|3758182|3766642|5058396|99548...,17|31|43|107|41|22|30|36|25|24|36|36|28|27|31|...,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
203387,003CQCD8POV7,4c2ae42150f97,1541366623,1,clickout item,3139558,BR,"Tatuí, Brazil",desktop,NaN,3139558|2499058|7252818|8117350|3994174|997034...,56|120|102|108|157|50|117,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
